Collect and extract features from wikipedia and wikimedia, must have internet connection

In [1]:
# import wikipedia
# print(wikipedia.summary("Wikipedia"))



from datetime import datetime
import time,sys
from pathlib import Path
import requests
import pickle
from urllib.request import Request, urlopen
import pandas as pd
from pytrends.request import TrendReq
from mediawiki import MediaWiki
import ast
import wptools

In [2]:
#set wikimedia agent
wikimedia = MediaWiki()

#parameters

wiki_farm=True #scrap from wikipedia
wikimedia_farm=True #scarp from wikimedia



In [3]:
test_term="5G"

Wikimedia scraper

In [4]:
%%time
def get_wikimedia_stats(name):
    wikimedia_dict_temp={}
    
    p = wikimedia.page(name)
    
    wikimedia_dict_temp['title']=p.title
    wikimedia_dict_temp['pageid']=p.pageid
    wikimedia_dict_temp['revision_id']=p.title
    wikimedia_dict_temp['parent_id']=p.parent_id
    # wikimedia_dict_temp['content']=p.content
    # wikimedia_dict_temp['html']=p.html
    # wikimedia_dict_temp['images']=p.images
    wikimedia_dict_temp['references']=len(p.references)
    wikimedia_dict_temp['categories']=len(p.categories)
    # wikimedia_dict_temp['coordinates']=p.coordinates
    wikimedia_dict_temp['redirects']=len(p.redirects)
    wikimedia_dict_temp['backlinks']=len(p.backlinks)
    wikimedia_dict_temp['langlinks']=len(p.langlinks)
    # wikimedia_dict_temp['summary']=p.summary
    wikimedia_dict_temp['sections']=len(p.sections)
    # wikimedia_dict_temp['logos']=p.logos
    # wikimedia_dict_temp['hatnotes']=p.hatnotes
    
    return wikimedia_dict_temp


get_wikimedia_stats(test_term)

CPU times: total: 0 ns
Wall time: 4.02 s


{'title': '5G',
 'pageid': '23475353',
 'revision_id': '5G',
 'parent_id': 1098494040,
 'references': 292,
 'categories': 8,
 'redirects': 21,
 'backlinks': 1088,
 'langlinks': 63,
 'sections': 53}

wiki stats

In [5]:
%%time
def get_wiki_stats(wiki_name, exclude_names_list):
    #try:
        # res = wikipedia.page(wiki_name)
        req = Request('https://xtools.wmflabs.org/articleinfo/en.wikipedia.org/Talk:' + wiki_name,
                      headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()

        tables = pd.read_html(webpage)  # here was the err
        name_dict = pd.concat(tables[0:5]).set_index(0).to_dict()[1]
        filtered_dict = {}
        for k in name_dict.keys():
            if k in exclude_names_list:
                continue
            else:
                filtered_dict[k] = name_dict[k]

        return filtered_dict
#     except Exception as e:
#         if "429" in str(e):
#             print("429 retry")
#             time.sleep(20)
#             return get_wiki_stats(wiki_name,exclude_names_list)
#         else:
#             print("Error: ",wiki_name)
#             print(e)
#             return {}
d=get_wiki_stats(test_term,[])   
d

CPU times: total: 15.6 ms
Wall time: 8.46 s


{'ID': '27705246',
 'Page size': '18,869 bytes',
 'Total edits': '768',
 'Editors': '201',
 'Page watchers': '618',
 'Pageviews (60 days)': '326',
 'Minor edits': '136  ·  (17.7%)',
 'IP edits': '56  ·  (7.3%)',
 'Bot edits': '83  ·  (10.8%)',
 '(Semi-)automated edits': '24',
 'Reverted edits': '30',
 'First edit': '2010-06-14 03:58  •  70.29.212.131  •  72',
 'Latest edit': '2022-07-09 17:18  •  99.197.202.188  •  2',
 'Max. text added': '2020-05-19 16:25  •  Lighteater  •  7,499',
 'Max. text deleted': '2019-12-25 04:51  •  Lowercase sigmabot III  •  -54,588',
 'Edits': 'Edits',
 'Average time between edits (days)': '5.7 days',
 'Average edits per user': '3.8',
 'Average edits per day': '0.2',
 'Average edits per month': '5.3',
 'Average edits per year': '63.6',
 'Edits in the past 24 hours': '0',
 'Edits in the past 7 days': '0',
 'Edits in the past 30 days': '2',
 'Edits in the past 365 days': '65',
 'Edits made by the top 10% of editors': '308  ·  (40.1%)',
 'Links': 'Links',
 'Li

wptools

In [1]:
#wptools example

# page = wptools.page(test_term, silent=False)
# page = wptools.page("Joe biden", silent=False)
# page.get_query()
# so = page.get_parse()

In [2]:
#so.data["assessments"]

# Load tech file

In [8]:
tech_df=pd.read_csv('https://docs.google.com/spreadsheets/d/1BZ_NP8PJbDsIXz4cbjSHlILQ8yBfy0vXcqm5Zw4fNoU/export?format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                  )
tech_list=tech_df.loc[tech_df['Emerging Tech'].notnull(),'Emerging Tech'].to_list()
#tech_list

# wikipedia historical views

In [14]:
def get_wiki_views_stats(wiki_name):
    url=f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/{wiki_name}/daily/20130101/20210109"
    req = Request(url,
                      headers={'User-Agent': 'Mozilla/5.0'})    
    webpage = urlopen(req).read()
    dict_str = webpage.decode("UTF-8")
    mydata = ast.literal_eval(dict_str)
    return mydata["items"]
l=get_wiki_views_stats("Bitcoin")

In [ ]:
# view_df=pd.read_csv("wiki_views.csv")

In [15]:
#set dict to collect tems data in case of crash
wiki_tech_dict={}
wiki_tech_df=tech_df[tech_df['wiki name'].notnull()]

In [ ]:
#for loop, collect wikiviews for each term
count=0

for index, row in wiki_tech_df.iloc[:,:].iterrows():
    name=row['wiki name']
    count+=1

    print(index,name)
    if name not in wiki_tech_df:
        if isinstance(name,str):
            try:
                wiki_dict=get_wiki_views_stats(name)
                wiki_tech_dict[name]=wiki_dict
                time.sleep(30)
            except Exception as e:
                print(e)
    else:
        print(index,name," exist")


In [ ]:
#save raw wikiviews to pickle
with open('wiki_views.pickle', 'wb') as handle:
    pickle.dump(wiki_tech_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#orgnize wikiviews date to unified format and merge all seperate technologies to a single file
i=0
dft=0
for key, value in wiki_tech_dict.items():
        df=pd.DataFrame(value)
#         print(df.loc[0,'timestamp'])
        df["date"]=pd.to_datetime(df["timestamp"], format='%Y%m%d00', errors='ignore')
        df=df[["date","views"]]
        df = df.rename(columns={'views': key})
        if i==0:
            dft=df.copy()
        else:
            dft= pd.merge(dft, df, on="date",how="outer")
        i+=1
#     if i==5:
#         display(dft)
#         break


In [ ]:
#merge terms and save to excel
dfft=dft.set_index("date").T
tech_replace_dict=tech_df[["Emerging Tech",'wiki name']].dropna().set_index('wiki name').to_dict()['Emerging Tech']
dfft=dfft.rename(index=tech_replace_dict)
dfft

In [ ]:
dfft.to_excel("wiki_views.xlsx")

# Wikipedia scrap

In [9]:
#load file of crashed in the middle of the scrap
tech_dict={}
with open('wiki.pickle', 'rb') as handle:
    tech_dict = pickle.load(handle)

In [ ]:
#create dict from wiki_name ti emerging tech name
tech_list_m=[]
for name in new_tech_list:
    alt_name=tech_df.loc[tech_df["Emerging Tech"]==name,"wiki name"].values[0]
    if isinstance(alt_name,str):
        tech_list_m.append(alt_name)
        print (name," ",type(alt_name))

In [ ]:
#for loop, collect wikimedia and wikipedia for each term
for index, row in tech_df.iloc[:,:].iterrows():
    try:
        count=0
        name=row['wiki name']
        if name not in tech_dict: 
            exclude_names_list = ['Page size', 'Total edits']

            wiki_tech_df=tech_df[tech_df['wiki name'].notnull()]
            wiki_dict = {}
            wikimedia_dict = {}
            trends_dict={}


            ##wikifarm
            if wiki_farm:  
                print(index,name)
                if isinstance(name,str):
                    wiki_dict=get_wiki_stats(name, exclude_names_list)
                    wiki_dict = {"wiki___" + str(key): val for key, val in wiki_dict.items()}

            ##wikimediafarm
            if wikimedia_farm:
                if isinstance(name,str):
                    wikimedia_dict=get_wikimedia_stats(name,)
                    wikimedia_dict = {"wikimedia___" + str(key): val for key, val in wikimedia_dict.items()}
                    time.sleep(5)
                    count+=1
                    if count==10:
                        count=0
                        time.sleep(30)

            wiki_dict.update(wikimedia_dict)
            tech_dict[name]=wiki_dict
    except Exception as e: 
        name=row['wiki name']
        print(index,name,e)







In [ ]:
#save to pickle
with open('wiki.pickle', 'wb') as handle:
    pickle.dump(tech_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# save raw data to excel

final_tech_df=pd.DataFrame.from_dict(tech_dict).T
tech_replace_dict=tech_df[["Emerging Tech",'wiki name']].dropna().set_index('wiki name').to_dict()['Emerging Tech']
final_tech_df=final_tech_df.rename(index=tech_replace_dict)
final_tech_df=final_tech_df.loc[final_tech_df['wiki___ID'].notnull(),:]
final_tech_df.to_excel('wiki.xlsx')

feature_df=final_tech_df.index.to_frame()
final_tech_df

,wiki___ID,wiki___Editors,wiki___Pageviews (60 days),wiki___Minor edits,wiki___IP edits,wiki___Bot edits,wiki___(Semi-)automated edits,wiki___Reverted edits,wiki___First edit,wiki___Latest edit,...,wikimedia___coordinates,wikimedia___redirects,wikimedia___backlinks,wikimedia___langlinks,wikimedia___summary,wikimedia___sections,wikimedia___logos,wikimedia___hatnotes,wiki___Max. text deleted,wiki___Page watchers
Hyper CD-ROM,25788971,11,25,5 · (27.8%),9 · (50%),6 · (33.3%),2,0,2010-01-13 05:01 • 76.66.197.17 • 80,2019-10-01 18:28 • SineBot • 287,...,None,[],"[1T-SRAM, 3D XPoint, 3D optical data storage, ...",{'ro': 'Hyper CD-ROM'},The Hyper CD-ROM is an optical data storage de...,"[Characteristics, References, External links]",[],[],NaN,NaN
5D DVD,23010806,8,11,1 · (12.5%),3 · (37.5%),1 · (12.5%),0,0,2009-05-30 04:20 • Grahamec • 99,2019-05-21 01:41 • 83.104.249.240 • 175,...,None,[Five dimensional discs],"[3D optical data storage, 5.1 Music Disc, ATA ...",{},A 5D DVD is an optical disc being developed by...,"[Background information, Advantages over curre...",[],[],2010-02-05 14:03 • TRS-80 • -7,NaN
3D Bioprinting,43319543,10,53,1 · (6.2%),2 · (12.5%),0 · (0%),2,0,2014-07-17 01:39 • Ceyockey • 477,2021-02-23 15:19 • GergelyGoat • 564,...,None,"[3D Bio-printing, 3D Bioprinting, Bio-printing...","[1978 in Ireland, 1978 in science, 2018 in sci...","{'ar': 'طباعة حيوية ثلاثية الأبعاد', 'bg': 'Тр...",Three dimensional (3D) bioprinting is the util...,"[Process, Pre-bioprinting, Bioprinting, Post-b...",[],[],2014-07-18 05:37 • 65.94.171.126 • -31,50
6G,65272061,3,22,2 · (40%),0 · (0%),2 · (40%),1,0,2020-09-09 20:19 • RMCD bot • 395,2020-12-10 22:46 • Dthomsen8 • 12,...,None,[6G (cellular network)],"[1G, 2G, 3G, 3G adoption, 4G, 5G, 5G NR, 6G, 7...","{'ar': 'جيل سادس (شبكات اتصال)', 'de': '6G', '...","In telecommunications, 6G is the sixth generat...","[Amplifier progress, Test satellite launch, Ex...",[],"[For other uses, see 6G (disambiguation).]",2020-09-10 20:22 • RMCD bot • -76,33
Second-generation biofuels,16118334,15,3,4 · (18.2%),6 · (27.3%),4 · (18.2%),1,0,2008-03-05 22:28 • Beagel • 50,2019-09-08 13:59 • Ashleystrenko • -232,...,None,"[Advanced Biofuels, Advanced biofuel, Advanced...","[Aviation biofuel, Avril Group, Bharat Petrole...","{'fa': 'نسل دوم زیست‌سوخت‌ها', 'hr': 'Biogoriv...","Second-generation biofuels, also known as adva...","[Introduction, Second-generation technology, T...",[],"[Main article: Gasification, Main article: Pyr...",2019-09-08 13:59 • Ashleystrenko • -232,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gallium Nitride semiconductor,9170393,17,10,6 · (18.2%),2 · (6.1%),2 · (6.1%),3,0,2007-01-28 17:41 • Beetstra • 32,2021-08-14 15:13 • RenatUK • -76,...,None,"[GaN, Gallium Nitride, Gallium(III) nitride]","[1989 in science, 6G (network), A. James Clark...","{'ar': 'نتريد الغاليوم', 'azb': 'نیترید قالیوم...",Gallium nitride (‹The template Gallium is bein...,"[Physical properties, Developments, LEDs, GaN ...",[https://upload.wikimedia.org/wikipedia/common...,"[This article is about Gallium nitride, the ch...",2021-08-14 15:13 • RenatUK • -76,108
Biosensor,3485099,25,8,13 · (31%),6 · (14.3%),13 · (31%),1,2,2005-12-20 23:57 • Dwayne Dibly • 356,2021-10-29 13:57 • MrOllie • -229,...,None,"[Biodetection, Biodetector, Biodetectors, Bios...","[2015 in science, 2021 in science, Agricultura...","{'ar': 'مستشعر حيوي', 'ca': 'Biosensor', 'cs':...","A biosensor is an analytical device, used for ...","[Biosensor system, Bioreceptors, Antibody/anti...",[],"[For the journals, see Biosensors (MDPI journa...",2008-01-23 11:31 • Benkeboy • -180,91
Floating Liquefied Natural Gas,31870883,4,5,1 · (20%),0 · (0%),4 · (80%),0,0,2011-05-24 17:04 • GregorB • 22,2021-06-05 11:13 • MalnadachBot • 32,...,None,"[FLNG, Floating Natural Gas, Floating liquefie...","[CNG carrier, China–Pakistan Economic Corridor...",{},A floating liquefied natural gas (FLNG) facili...,"[History, Current project

In [ ]:
#fill missing data with 0

In [ ]:
for name in list(set(tech_list)-set(final_tech_df.index) ):
    final_tech_df.loc[name]=0

In [ ]:
final_tech_df=final_tech_df.fillna(0)

In [ ]:
final_tech_df.loc[final_tech_df['wiki___Minor edits']=='0','wiki___Minor edits']="0  ·  (0.0%)"

Wikipedia

In [ ]:
#create wikipedia features from raw data
feature_df['wiki_N_of_total_edits']=final_tech_df['wiki___Minor edits'].astype(str).str.replace(",","").str.split("·",0).str[0].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___IP edits'].astype(str).str.replace(",","").str.split("·",0).str[0].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___Bot edits'].astype(str).str.replace(",","").str.split("·",0).str[0].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___(Semi-)automated edits'].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___Reverted edits'].astype('int64')

feature_df['wiki_N_of_editors']=final_tech_df['wiki___Editors'].astype('int64')

feature_df['wiki_N_of_files']=final_tech_df['wiki___Files'].astype('int64')

feature_df['wiki_N_of_categories']=final_tech_df['wiki___Categories'].astype('int64')

feature_df['wiki_N_of_external_links']=final_tech_df['wiki___External links'].astype('int64')

feature_df['wiki_N_of_income_links']=final_tech_df['wiki___Links to this page'].astype('int64')

feature_df['wiki_N_of_redirects']=final_tech_df['wiki___Redirects'].astype('int64')

feature_df['wiki_N_of_outcome_links']=final_tech_df['wiki___Links from this page'].astype('int64')

feature_df['wiki_N_of_pageviewes']=final_tech_df['wiki___Pageviews (60 days)'].astype('int64')

feature_df['wiki_N_of_averege time btween_edits']=final_tech_df['wiki___Average time between edits (days)'].str.split(" ",0).str[0]

feature_df['wiki_N_of_averege _edits_per_day']=final_tech_df['wiki___Average edits per day'].astype('float')

feature_df['wiki_N_of_averege _edits_made by top 10% editors']=final_tech_df['wiki___Edits made by the top 10% of editors'].str.split("·",0).str[0]

feature_df['wiki_N_of_Characters']=final_tech_df['wiki___Characters'].astype('int64')

feature_df['wiki_N_of_sections']=final_tech_df['wiki___Sections'].astype('int64')

feature_df['wiki_N_of_references']=final_tech_df['wiki___References'].astype('int64')

feature_df['wiki_N_of_Unique references']=final_tech_df['wiki___Unique references'].astype('int64')

feature_df['wiki_N_of_bot_edits']=final_tech_df['wiki___Bot edits'].astype(str).str.split("·",0).str[0].astype('int64')

feature_df['wiki_N_of_a_major_edits']=final_tech_df['wiki___Minor edits'].astype(str).str.replace(",","").str.split("·",0).str[0].astype('int64').fillna(0)-feature_df['wiki_N_of_total_edits'].fillna(0).astype('int64')

feature_df['wiki_N_of_pagewatchers']=final_tech_df['wiki___Page watchers']

feature_df['wiki_N_of_languages']=len(final_tech_df["wikimedia___langlinks"])

In [ ]:
for name in feature_df.columns:
    data_path=f"features/wiki/features/{name}/"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    print(name,data_path)
    feature_df[name].to_frame().to_excel(f"{data_path}{name}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")

In [ ]:
for name in final_tech_df.columns:
    data_path=f"features/wiki/fields/{name}/"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    print(name,data_path)
    final_tech_df[name].to_frame().to_excel(f"{data_path}{name}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")